In [1]:
import pandas as pd
import datetime
import re
import numpy as np
from pipeline import ingest, transform, utils
from config import config

In [2]:
def clean_columns(x):
    """ Clean columns by replacing non-ascii and 'bad' characters """
    x = str(x).replace('−', '-')
    x = re.sub(r"[^().+\-:\d]", '', x)
    return x

In [3]:
datetime.date.today()

datetime.date(2025, 7, 30)

In [4]:
edinburgh_columns = [
    'date', 
    'sunrise', 
    'sunset', 
    'daylength', 
    'daylength_diff', 
    'astro_twilight_start', 
    'astro_twilight_end', 
    'nautical_twilight_start', 
    'nautical_twilight_end', 
    'civil_twilight_start', 
    'civil_twilight_end', 
    'solar_noon_time', 
    'solar_noon_dist'
    ]

In [5]:
strathspey_columns = [
    'date',
    'temp_c_mean',
    'temp_c_min',
    'temp_c_max',
    'rain_mm',
    'pressure_mb_am',
    'pressure_mb_pm',
    'wind_mean_mph',
    'wind_max_mph',
    'wind_direction',
    'sun_hours'
]

In [6]:
def drop_empty_rows(df, thresh=0.5):
    """
    Drops a row if at least "thresh%" is missing.
    """
    return df.dropna(thresh=int(thresh*len(df.columns))).reset_index(drop=True)

In [7]:
def square_number(x):
    return x**2

In [8]:
def power_number(x, n=2):
    return x**n

In [9]:
def load_sheets(file_path):
    """ Load Excel workbook with all sheets"""
    return pd.read_excel(file_path, sheet_name=None)

In [10]:
def get_month_year(yymm, century=20):
    """ Get month and year from YYMM string """
    # print(f"YYMM: {k}")
    month = int(yymm[2:])
    year = century*100 + int(yymm[:2])
    return month, year

In [11]:
def drop_rows(df, n):
    """ Drop the leading n rows from a DataFrame """
    # drop first n rows and any empties
    return df.drop(index=range(n)).reset_index(drop=True)

In [12]:
def clean_daytime_sheets(sheets, column_names):
    """ Processing all sheets and concatenating them into a single DataFrame """
    all_sheets = []
    for k in sheets.keys():
        month, year = get_month_year(k)   
        df = sheets[k]
        
        # change column names
        df.columns = column_names
        # or
        # df = df.drop(df.index[[0, 1]])
        
        # drop first n rows and any empties
        df = drop_rows(df, 2)
        df = drop_empty_rows(df).reset_index(drop=True)
        
        # create sheet flag
        # df['sheet'] = k 
        
        # clean columns
        for col in df.columns:
            if col != 'date':
                df[col] = df[col].apply(clean_columns)
        for col in ['sunrise','sunset','solar_noon_time']:
            df[col] = df[col].str.split('(', expand=True).loc[:,0] 
            
        # update date
        df["date"] = df["date"].apply(lambda x: datetime.date(year, month, int(x)))
        all_sheets.append(df)
        
    return pd.concat(all_sheets, ignore_index=True)

In [13]:
edin_sheets = load_sheets('data/Edinburgh-daytime.xlsx')
edin_df = clean_daytime_sheets(edin_sheets, edinburgh_columns)
edin_df.head(10)

,date,sunrise,sunset,daylength,daylength_diff,astro_twilight_start,astro_twilight_end,nautical_twilight_start,nautical_twilight_end,civil_twilight_start,civil_twilight_end,solar_noon_time,solar_noon_dist
0,2011-12-01,08:18,15:44,07:25:45,-2:37,06:00:00,18:02:00,06:45:00,17:17:00,07:33:00,16:29:00,12:01,147.52
1,2011-12-02,08:20,15:43,07:23:13,-2:31,06:02:00,18:01:00,06:46:00,17:16:00,07:34:00,16:28:00,12:02,147.495
2,2011-12-03,08:21,15:42,07:20:48,-2:25,06:03:00,18:01:00,06:48:00,17:16:00,07:36:00,16:28:00,12:02,147.471
3,2011-12-04,08:23,15:41,07:18:29,-2:18,06:04:00,18:00:00,06:49:00,17:15:00,07:37:00,16:27:00,12:02,147.447
4,2011-12-05,08:24,15:41,07:16:17,-2:12,06:05:00,18:00:00,06:50:00,17:15:00,07:39:00,16:27:00,12:03,147.425
5,2011-12-06,08:26,15:40,07:14:11,-2:05,06:06:00,18:00:00,06:52:00,17:14:00,07:40:00,16:26:00,12:03,147.403
6,2011-12-07,08:27,15:40,07:12:12,-1:58,06:07:00,17:59:00,06:53:00,17:14:00,07:41:00,16:26:00,12:04,147.381
7,2011-12-08,08:29,15:39,07:10:20,-1:51,06:09:00,17:59:00,06:54:00,17:14:00,07:42:00,16:25:00,12:04,147.361
8,2011-12-09,08:30,15:39,07:08:36,-1:44,06:10:00,17:59:00,06:55:00,17:14:00,07:44:00,16:25:00,12:04,147.342
9,2011-12-10,08:31,15:38,07:06:58,-1:37,06:11:00,17:59:00,06:56:00,17:14:00,07:45:00,16:25:00,12:05,147.323


In [14]:
# define snow column
def define_snow(df):
    """ Define snow column based on rain_mm """
    df['snow'] = np.where(df['rain_mm'].astype(str).str.endswith('s'),
                        df['rain_mm'], 
                        None)
    df['rain_mm'] = np.where(df['rain_mm'].astype(str).str.endswith('s'),
                        None, 
                        df['rain_mm'])
    return df

In [15]:
def define_date(df, year, month):
    df["date"] = df["date"].apply(lambda x: datetime.date(year, month, int(x)))
    return df

In [16]:
def clean_weather_sheets(sheets, column_names):
    """ Processing all sheets and concatenating them into a single DataFrame """
    all_sheets = []
    for k in sheets.keys():
        # print(f"YYMM: {k}")
        month, year = get_month_year(k)
        df = sheets[k]
        
        # # change column names
        df.columns = column_names
        
        # # drop first five rows and any empties
        df = drop_rows(df, 5)
        df = drop_empty_rows(df).reset_index(drop=True)
        # # or
        # # df = df.drop(df.index[[0, 1]])
        
        # define snow column
        df = define_snow(df)
        
        # # clean columns
        for col in df.columns:
            if col not in ['date', 'wind_direction']:
                df[col] = df[col].apply(clean_columns)
        # update date
        df = define_date(df, year, month)
        all_sheets.append(df)
        
    return pd.concat(all_sheets, ignore_index=True)

In [17]:
strat_sheets = load_sheets('data/Strathspey-weather.xlsx')
strat_df = clean_weather_sheets(strat_sheets, strathspey_columns)
strat_df.head(10)

,date,temp_c_mean,temp_c_min,temp_c_max,rain_mm,pressure_mb_am,pressure_mb_pm,wind_mean_mph,wind_max_mph,wind_direction,sun_hours,snow
0,2011-12-01,3.6,-0.4,4.6,0,995,1004,4.1,36,SSW,2.7,
1,2011-12-02,2.6,-1.5,7.5,5.5,1003,983,6.5,36,SSW,1.8,
2,2011-12-03,2.4,0.4,4.4,,986,988,9.5,46,SSW,0.75,6.0
3,2011-12-04,-1.6,-0.9,0.4,,988,992,4.5,22,SSW,1.66,3.7
4,2011-12-05,-1.6,-4.2,0.2,,994,998,2.8,17,SSW,1.08,2.9
5,2011-12-06,-2.7,-2.8,1.2,,998,992,37,19,WSW,2.92,5.1
6,2011-12-07,0.2,-5.3,3.8,,989,1005,6.1,39,SSW,0.15,8.6
7,2011-12-08,2.4,-0.8,6.7,14.6,979,981,15.9,62,SSW,0.72,
8,2011-12-09,-0.8,0.9,1.4,,1000,1007,6.7,34,WSW,0.75,2.6
9,2011-12-10,-3.2,-8,3.5,,1008,1001,1,10,SSE,1.46,0.5


In [19]:
edin_df.isnull().sum()

date                       0
sunrise                    0
sunset                     0
daylength                  0
daylength_diff             0
astro_twilight_start       0
astro_twilight_end         0
nautical_twilight_start    0
nautical_twilight_end      0
civil_twilight_start       0
civil_twilight_end         0
solar_noon_time            0
solar_noon_dist            0
dtype: int64

In [21]:
strat_df.isnull().sum()

date              0
temp_c_mean       0
temp_c_min        0
temp_c_max        0
rain_mm           0
pressure_mb_am    0
pressure_mb_pm    0
wind_mean_mph     0
wind_max_mph      0
wind_direction    0
sun_hours         0
snow              0
dtype: int64

In [22]:
pd.ExcelFile('data/Edinburgh-daytime.xlsx').sheet_names

['1112',
 '1201',
 '1202',
 '1203',
 '1204',
 '1205',
 '1206',
 '1207',
 '1208',
 '1209',
 '1210',
 '1211',
 '1212',
 '1301']

In [23]:
pd.ExcelFile('data/Strathspey-weather.xlsx').sheet_names

['1112',
 '1201',
 '1202',
 '1203',
 '1204',
 '1205',
 '1206',
 '1207',
 '1208',
 '1209',
 '1210',
 '1211',
 '1212',
 '1301']

In [ ]:
def merge_datasets(df1, df2):
    """ Merge two DataFrames on 'date' column """
    return df1.merge(df2, on='date', how='inner')

In [ ]:
merged = merge_datasets(edin_df, strat_df)
merged.head(10)

,date,sunrise,sunset,daylength,daylength_diff,astro_twilight_start,astro_twilight_end,nautical_twilight_start,nautical_twilight_end,civil_twilight_start,...,temp_c_min,temp_c_max,rain_mm,pressure_mb_am,pressure_mb_pm,wind_mean_mph,wind_max_mph,wind_direction,sun_hours,snow
0,2011-12-01,08:18,15:44,07:25:45,-2:37,06:00:00,18:02:00,06:45:00,17:17:00,07:33:00,...,-0.4,4.6,0,995,1004,4.1,36,SSW,2.7,
1,2011-12-02,08:20,15:43,07:23:13,-2:31,06:02:00,18:01:00,06:46:00,17:16:00,07:34:00,...,-1.5,7.5,5.5,1003,983,6.5,36,SSW,1.8,
2,2011-12-03,08:21,15:42,07:20:48,-2:25,06:03:00,18:01:00,06:48:00,17:16:00,07:36:00,...,0.4,4.4,,986,988,9.5,46,SSW,0.75,6.0
3,2011-12-04,08:23,15:41,07:18:29,-2:18,06:04:00,18:00:00,06:49:00,17:15:00,07:37:00,...,-0.9,0.4,,988,992,4.5,22,SSW,1.66,3.7
4,2011-12-05,08:24,15:41,07:16:17,-2:12,06:05:00,18:00:00,06:50:00,17:15:00,07:39:00,...,-4.2,0.2,,994,998,2.8,17,SSW,1.08,2.9
5,2011-12-06,08:26,15:40,07:14:11,-2:05,06:06:00,18:00:00,06:52:00,17:14:00,07:40:00,...,-2.8,1.2,,998,992,37,19,WSW,2.92,5.1
6,2011-12-07,08:27,15:40,07:12:12,-1:58,06:07:00,17:59:00,06:53:00,17:14:00,07:41:00,...,-5.3,3.8,,989,1005,6.1,39,SSW,0.15,8.6
7,2011-12-08,08:29,15:39,07:10:20,-1:51,06:09:00,17:59:00,06:54:00,17:14:00,07:42:00,...,-0.8,6.7,14.6,979,981,15.9,62,SSW,0.72,
8,2011-12-09,08:30,15:39,07:08:36,-1:44,06:10:00,17:59:00,06:55:00,17:14:00,07:44:00,...,0.9,1.4,,1000,1007,6.7,34,WSW,0.75,2.6
9,2011-12-10,08:31,15:38,07:06:58,-1:37,06:11:00,17:59:00,06:56:00,17:14:00,07:45:00,...,-8,3.5,,1008,1001,1,10,SSE,1.46,0.5


In [ ]:
merged.to_csv('outputs/scottish_weather_data.csv', index=False)